In [1]:
import json 
import pandas as pd
import numpy as np
import time
from scipy.sparse import dok_matrix
import itertools
from sklearn import preprocessing

In [2]:
data = json.load(open('./mpd.v1/data/mpd.slice.0-999.json'))
data['info']

{'generated_on': '2017-12-03 08:41:42.057563',
 'slice': '0-999',
 'version': 'v1'}

In [3]:
lists_data = []

for i in range(0,10):
    for x in data['playlists'][i]['tracks']:
        lists_data.append([data['playlists'][i]['name'],data['playlists'][i]['num_albums'], data['playlists'][i]['num_artists'],data['playlists'][i]['num_followers'],data['playlists'][i]['num_tracks'], data['playlists'][i]['pid'], x['pos'], x['artist_name'], x['track_name'], x['duration_ms'], x['album_name']])


df = pd.DataFrame(lists_data, columns = ['play_list_name', 'num_albums', 'num_artists', 'num_followers', 'num_tracks', 'pid', 'song_pos', 'song_artist', 'song_name', 'song_duration', 'song_album'])

In [4]:
df.head()

,play_list_name,num_albums,num_artists,num_followers,num_tracks,pid,song_pos,song_artist,song_name,song_duration,song_album
0,Throwbacks,47,37,1,52,0,0,Missy Elliott,Lose Control (feat. Ciara & Fat Man Scoop),226863,The Cookbook
1,Throwbacks,47,37,1,52,0,1,Britney Spears,Toxic,198800,In The Zone
2,Throwbacks,47,37,1,52,0,2,Beyoncé,Crazy In Love,235933,Dangerously In Love (Alben für die Ewigkeit)
3,Throwbacks,47,37,1,52,0,3,Justin Timberlake,Rock Your Body,267266,Justified
4,Throwbacks,47,37,1,52,0,4,Shaggy,It Wasn't Me,227600,Hot Shot


In [8]:
# listas de listas de canciones 
listas = {}
cont = 0 

for i in range(0,10):
    spot = []
    for x in data['playlists'][i]['tracks']:
        cont+=1
        spot.append(x['track_name'])

        if cont == len(data['playlists'][i]['tracks']):
            listas[data['playlists'][i]['pid']] = spot       
            cont = 0 
            spot = []


In [9]:
print(len(df.pid.unique()))
print(len(df.song_name.unique()))

10
502


In [10]:
inicio = time.time()

songs_unique = df.song_name.unique()

le = preprocessing.LabelEncoder()
enc_songs = le.fit(songs_unique)
size = len(le.classes_)
matriz = dok_matrix((size,size))

pid_unique = df.pid.unique()

for (i,j) in itertools.combinations(songs_unique,2):
    for k in pid_unique:
        if j in listas[k] and i in listas[k] and i!=j:
            matriz[le.transform([i,j])] += 1               
    
fin = time.time()

print('tiempo demorado: {} segs'.format(fin - inicio))

tiempo demorado: 172.13251495361328 segs


In [11]:
result = {}

for x in range(0,matriz.shape[0]):
    for y in range(0, matriz.shape[0]):
        if matriz[x,y] > 0 and x <y:
            result[(x,y)] = matriz[x,y]


In [ ]:
df_result = pd.DataFrame(result, columns = ['Target', 'Source', 'Weight'])

df_result.to_csv('spotify_song__graph.csv', sep=';', index=False)

In [12]:
result

{(0, 1): 125.0,
 (0, 2): 125.0,
 (0, 3): 125.0,
 (0, 4): 125.0,
 (0, 5): 125.0,
 (0, 6): 125.0,
 (0, 7): 125.0,
 (0, 8): 125.0,
 (0, 9): 125.0,
 (0, 10): 125.0,
 (0, 11): 125.0,
 (0, 12): 125.0,
 (0, 13): 125.0,
 (0, 14): 125.0,
 (0, 15): 125.0,
 (0, 16): 125.0,
 (0, 17): 125.0,
 (0, 18): 125.0,
 (0, 19): 125.0,
 (0, 20): 125.0,
 (0, 21): 125.0,
 (0, 22): 125.0,
 (0, 23): 125.0,
 (0, 24): 125.0,
 (0, 25): 125.0,
 (0, 26): 125.0,
 (0, 27): 125.0,
 (0, 28): 125.0,
 (0, 29): 125.0,
 (0, 30): 125.0,
 (0, 31): 125.0,
 (0, 32): 125.0,
 (0, 33): 125.0,
 (0, 34): 125.0,
 (0, 35): 125.0,
 (0, 36): 125.0,
 (0, 37): 125.0,
 (0, 38): 125.0,
 (0, 39): 125.0,
 (0, 40): 125.0,
 (0, 41): 125.0,
 (0, 42): 125.0,
 (0, 43): 125.0,
 (0, 44): 125.0,
 (0, 45): 125.0,
 (0, 46): 125.0,
 (0, 47): 125.0,
 (0, 48): 125.0,
 (0, 49): 125.0,
 (0, 50): 125.0,
 (0, 51): 125.0,
 (0, 52): 125.0,
 (0, 53): 125.0,
 (0, 54): 125.0,
 (0, 55): 125.0,
 (0, 56): 125.0,
 (0, 57): 125.0,
 (0, 58): 125.0,
 (0, 59): 125.0,
 (0, 6

In [15]:
list(itertools.combinations(songs_unique,2))

[('Lose Control (feat. Ciara & Fat Man Scoop)', 'Toxic'),
 ('Lose Control (feat. Ciara & Fat Man Scoop)', 'Crazy In Love'),
 ('Lose Control (feat. Ciara & Fat Man Scoop)', 'Rock Your Body'),
 ('Lose Control (feat. Ciara & Fat Man Scoop)', "It Wasn't Me"),
 ('Lose Control (feat. Ciara & Fat Man Scoop)', 'Yeah!'),
 ('Lose Control (feat. Ciara & Fat Man Scoop)', 'My Boo'),
 ('Lose Control (feat. Ciara & Fat Man Scoop)', 'Buttons'),
 ('Lose Control (feat. Ciara & Fat Man Scoop)', 'Say My Name'),
 ('Lose Control (feat. Ciara & Fat Man Scoop)',
  'Hey Ya! - Radio Mix / Club Mix'),
 ('Lose Control (feat. Ciara & Fat Man Scoop)', 'Promiscuous'),
 ('Lose Control (feat. Ciara & Fat Man Scoop)',
  'Right Where You Want Me - Radio Edit Version'),
 ('Lose Control (feat. Ciara & Fat Man Scoop)', 'Beautiful Soul'),
 ('Lose Control (feat. Ciara & Fat Man Scoop)', "Leavin'"),
 ('Lose Control (feat. Ciara & Fat Man Scoop)', 'Me & U'),
 ('Lose Control (feat. Ciara & Fat Man Scoop)', 'Ice Box'),
 ('Lose C